In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# set working directory -> you must set the path into which you have uploaded the zipped file
# this is required in the case of colab - not in local mode
%cd /content/drive/My\ Drive/DMTM

/content/drive/My Drive/DMTM


In [ ]:
# list content of drive
%ls

TestParameter.txt  TestTransaction.txt


In [ ]:
# READ Transaction File 
transactions = []

fileData = open('TestTransaction.txt', "r+").readlines()
  
# remove all whitespaces, all newline chars, trailing commas that have no input after it
arrTransact = [line.replace(" ", "").rstrip("\n").rstrip(",") for line in fileData]

transactions = [list(map(int, line.split(","))) for line in arrTransact]

transactions

[[10, 50, 30, 40], [1, 2, 4, 6, 8, 9, 10], [3, 4, 5], [7, 9, 20]]

In [ ]:
# READ Parameter file 
MIS= {}
SDC= 0 

mis_rest_val = 0.0

fileData = open('TestParameter.txt', "r+").readlines()
arrParams = [line.replace(" ", "").rstrip("\n") for line in fileData]

# now we must read exactly-> MIS(n)=<float_value> and MIS(rest)=<float_value> and SDC=<float_value>
for param in arrParams:
    if param.find("MIS") > -1:
        # get indices
        key_start_idx = param.find("(")
        key_end_idx = param.find(")")
        val_start_idx = param.find("=")
        
        # extract & record dict keys and vals 
        key = param[key_start_idx + 1 : key_end_idx]
        val = float((param[val_start_idx + 1 :]))
        
        if key != 'rest':
            MIS[int(key)] = val
        else:
            # if REST -> we have to add this value for keys not in dict already later on 
            mis_rest_val = val
                    

    if param.find("SDC=") > -1:
        SDC = param[param.find("=") + 1 :]

        # assert if float works
        try:
            SDC = float(SDC)
        except ValueError:
            raise Exception("Error converting SDC value")

# now populate mis dict with pending item values using mis_rest_val
for t in transactions:
  for item in t:
    if item not in MIS.keys():
      MIS[int(item)] = mis_rest_val

MIS

{1: 0.02,
 2: 0.04,
 3: 0.01,
 4: 0.01,
 5: 0.01,
 6: 0.01,
 7: 0.04,
 8: 0.01,
 9: 0.01,
 10: 0.01,
 20: 0.01,
 30: 0.01,
 40: 0.01,
 50: 0.01}

In [ ]:
# Calculate M <- sort(I, MS) 
# where I == itemset in each transaction record
M = []

for t in transactions:
  t.sort()
  tmpD = []
  for item in t:
    tmpD.append((item,MIS[item]))

  tmpD.sort(key=lambda tup: tup[1])  # sorts in place
  M.append(tmpD)
  # sort tmpD based on tuple values
    
M

[[(10, 0.01), (30, 0.01), (40, 0.01), (50, 0.01)],
 [(4, 0.01),
  (6, 0.01),
  (8, 0.01),
  (9, 0.01),
  (10, 0.01),
  (1, 0.02),
  (2, 0.04)],
 [(3, 0.01), (4, 0.01), (5, 0.01)],
 [(9, 0.01), (20, 0.01), (7, 0.04)]]

In [ ]:
import operator

# Calc L <- init-pass(M, T)
# basically counts of 1-itemsets in each transaction 
L = {}

# order indexer 
order = 1

for t in M:
  for tpl_item in t:
    item = tpl_item[0]
    item_mis = tpl_item[1]

    if item not in L.keys():
      L[item] = {
          'mis': item_mis,
          'count': 1,
          'order': order
      }

      order += 1
    else:
      L[item]['count'] += 1 



In [ ]:
# calculate F1 ==> Frequent itemsets of size 1
n = len(transactions)

F = {}
F['1'] = {}
F['1']['items'] = []

for i in L:
  if ( L[i]['count'] / n ) >= L[i]['mis'] :
    F['1']['items'].append(i)

F

{'1': {'items': [10, 30, 40, 50, 4, 6, 8, 9, 1, 2, 3, 5, 20, 7]}}

In [ ]:
import itertools 

def fn_Sup(val):
  return val/n 

def level2_candidate_gen(L, SDC):
  print('here')
  C2 = []

  for i in sorted(L, key = lambda name: L[name]['order']):

    if ( L[i]['count'] / n ) >= L[i]['mis'] :
      for h in sorted(L, key = lambda name: L[name]['order']):

        if L[h]['order'] > L[i]['order']:
          sdc_diff = fn_Sup(L[h]['count']) -  fn_Sup(L[i]['count'])
          
          if ( (L[h]['count']/n) >= L[i]['mis'] ) and ( abs(sdc_diff) <= SDC ) :
            C2.append({'items': [i,h], 'counts': 0, 'tailcounts': 0})
            
  return C2


def MScandidate_gen(Fk, SDC, k):
  Ck = []

  for f1 in Fk:
    for f2 in Fk:
      #local var
      c =[]

      # doing this step to match pseudo code in textbook
      f1['items'].sort()
      f2['items'].sort()

      #print('f1 =>', f1['items'])
      #print('f2 =>', f2['items'])

      if set(f1['items']) != set(f2['items']):
        ik = f1['items'][-1]
        ik_dash = f2['items'][-1]
        sdc_diff = fn_Sup(L[ik]['count']) -  fn_Sup(L[ik_dash]['count'])

        #print('sdc',abs(sdc_diff) <= SDC)
        if (ik < ik_dash) and ( abs(sdc_diff) <= SDC ):
          c = f1['items']+f2['items'][-1:]
          Ck.append({'items':c, 'counts': 0, 'tailcounts': 0})
          
          
          for s in list(itertools.combinations(c, k)):
            if set([c[0]]).issubset(s) or ( L[c[0]]['mis'] ==  L[c[1]]['mis'] ) : 
              is_subset = False
              for f in Fk:
                if set(s).issubset(f['items']): 
                  is_subset = True
                  break;

              if is_subset == False:
                Ck.pop()
                break

  return Ck



In [ ]:
k = 2 

C = {}

# loop K until the end of freq itemset iteration doesnt return empty set 
while len(F[str(k-1)]) > 0:
  if k == 2:
    C[2] = level2_candidate_gen(L, SDC)
  else:
    C[k] = MScandidate_gen(F[str(k-1)], SDC, k-1)
    print(C[k])

  ck_counts = []
  ck_tail_counts = []

  for t in transactions:
    for c in C[k]:
      if set(c['items']).issubset(t):
        c['counts'] += 1
      if set(c['items'][1:]).issubset(t):
        c['tailcounts'] += 1

  F[str(k)] = []
  for c in C[k]:
    if ( c['counts'] / n) >= L[c['items'][0]]['mis']:
      F[str(k)].append(c)
      
  print(k)
  k+= 1

  

 

here
2
[{'items': [30, 40, 50], 'counts': 0, 'tailcounts': 0}, {'items': [30, 40, 50], 'counts': 0, 'tailcounts': 0}, {'items': [4, 9, 10], 'counts': 0, 'tailcounts': 0}, {'items': [4, 9, 10], 'counts': 0, 'tailcounts': 0}, {'items': [1, 6, 8], 'counts': 0, 'tailcounts': 0}, {'items': [1, 6, 8], 'counts': 0, 'tailcounts': 0}, {'items': [1, 6, 8], 'counts': 0, 'tailcounts': 0}, {'items': [2, 6, 8], 'counts': 0, 'tailcounts': 0}, {'items': [2, 6, 8], 'counts': 0, 'tailcounts': 0}, {'items': [2, 6, 8], 'counts': 0, 'tailcounts': 0}, {'items': [1, 2, 8], 'counts': 0, 'tailcounts': 0}, {'items': [1, 2, 6], 'counts': 0, 'tailcounts': 0}, {'items': [1, 2, 6], 'counts': 0, 'tailcounts': 0}, {'items': [1, 2, 8], 'counts': 0, 'tailcounts': 0}, {'items': [1, 2, 8], 'counts': 0, 'tailcounts': 0}]
3
[{'items': [1, 2, 6, 8], 'counts': 0, 'tailcounts': 0}, {'items': [1, 2, 6, 8], 'counts': 0, 'tailcounts': 0}, {'items': [1, 2, 6, 8], 'counts': 0, 'tailcounts': 0}, {'items': [1, 2, 6, 8], 'counts': 0,

In [ ]:
# HERE we will print this output for u for F
result = ''
for i in F: 
  if int(i) == 1:
    if len( F[i]['items']) > 0:
      lgth = len(F[i]['items'])
      result += f'(Length-{i} { lgth } '
      result += '\n\t('
      result += ')\n\t('.join(map(str, F[i]['items']))
      result += ')\n'
      result += ') \n' # close length

  else:
    if len(F[i]) > 0:
      result += f'(Length-{i} {len(F[i])}'
      result += ''

      for pair in F[i]:
        result += '\n\t('+ ' '.join(map(str, pair['items']))
        # for elem in pair['items']: 
        #   result += f'{elem}'
        result += ')'


    # for pair in F[i]:
    #   print(pair)
      
    result += ') \n'
print(result)



(Length-1 14 
	(10)
	(30)
	(40)
	(50)
	(4)
	(6)
	(8)
	(9)
	(1)
	(2)
	(3)
	(5)
	(20)
	(7)
) 
(Length-2 14
	(4 10)
	(9 10)
	(30 40)
	(30 50)
	(40 50)
	(4 9)
	(6 8)
	(1 6)
	(2 6)
	(1 8)
	(2 8)
	(1 2)
	(3 5)
	(7 20)) 
(Length-3 15
	(30 40 50)
	(30 40 50)
	(4 9 10)
	(4 9 10)
	(1 6 8)
	(1 6 8)
	(1 6 8)
	(2 6 8)
	(2 6 8)
	(2 6 8)
	(1 2 8)
	(1 2 6)
	(1 2 6)
	(1 2 8)
	(1 2 8)) 
(Length-4 18
	(1 2 6 8)
	(1 2 6 8)
	(1 2 6 8)
	(1 2 6 8)
	(1 2 6 8)
	(1 2 6 8)
	(1 2 6 8)
	(1 2 6 8)
	(1 2 6 8)
	(1 2 6 8)
	(1 2 6 8)
	(1 2 6 8)
	(1 2 6 8)
	(1 2 6 8)
	(1 2 6 8)
	(1 2 6 8)
	(1 2 6 8)
	(1 2 6 8)) 
) 

